In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications import resnet50
from keras.models import Model, load_model
from keras.layers import Input

import matplotlib.pyplot as plt
import cv2

2024-02-04 21:10:41.408410: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-04 21:10:41.461013: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 21:10:41.461057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 21:10:41.462089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 21:10:41.469721: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-04 21:10:41.470339: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
for dirname, _, filenames in os.walk('./data'):
    print(dirname)

./data
./data/cell_images
./data/cell_images/cell_images
./data/cell_images/cell_images/Parasitized
./data/cell_images/cell_images/Uninfected
./data/Parasitized
./data/Uninfected


In [4]:
def data_prep(parasitized, uninfected):
    
    parasitized = ["./data/Parasitized" + '/' +  parasitize for parasitize in parasitized]
    uninfected = ["./data/Uninfected" + '/' +  uninfect for uninfect in uninfected]

    labels = len(parasitized) * ['parasitized'] + len(uninfected) * ['uninfected']
    data = parasitized + uninfected

    return pd.DataFrame({'Image_Path': data , 'Labels': labels})

In [5]:
df = data_prep(os.listdir("./data/Parasitized/"), os.listdir("./data/Uninfected/"))

In [6]:
print("Shape of the DataFrame:", df.shape)
print("\nHead of the DataFrame:")
print(df.head())
print("\nTail of the DataFrame:")
print(df.tail())


Shape of the DataFrame: (27558, 2)

Head of the DataFrame:
                                          Image_Path       Labels
0  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
1  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
2  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
3  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
4  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized

Tail of the DataFrame:
                                              Image_Path      Labels
27553  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27554  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27555  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27556  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27557  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected


In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function to load and preprocess images
def load_images(file_paths, target_size=(64, 64)):
    images = []
    for path in file_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        images.append(img)
    return np.array(images)

# Load and preprocess images
X = load_images(df['Image_Path'])
y = df['Labels']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN model
model.fit(X_train, (y_train == 'parasitized').astype(int), epochs=5, batch_size=32, validation_split=0.2)

# Extract features from the CNN model
cnn_features_train = model.predict(X_train)
cnn_features_test = model.predict(X_test)

# Reshape features for SVM
cnn_features_train_flat = cnn_features_train.reshape(cnn_features_train.shape[0], -1)
cnn_features_test_flat = cnn_features_test.reshape(cnn_features_test.shape[0], -1)

# List of kernel types
kernels = ['linear', 'poly', 'rbf']

# Train and evaluate SVM models for each kernel
for kernel_type in kernels:
    # Build an SVM model
    svm_model = SVC(kernel=kernel_type)
    svm_model.fit(cnn_features_train_flat, (y_train == 'parasitized'))

    # Predict using the SVM model
    svm_predictions = svm_model.predict(cnn_features_test_flat)

    # Evaluate the SVM model for each kernel
    accuracy = accuracy_score((y_test == 'parasitized'), svm_predictions)
    print(f"Accuracy of the SVM model (kernel: {kernel_type}): {accuracy*100}")

Epoch 1/5


2024-02-04 21:10:55.499364: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 216711168 exceeds 10% of free system memory.


552/552 [==============================] - 74s 132ms/step - loss: 15.4203 - accuracy: 0.6663 - val_loss: 1.2545 - val_accuracy: 0.6615
Epoch 2/5
552/552 [==============================] - 79s 144ms/step - loss: 0.6688 - accuracy: 0.7842 - val_loss: 0.6272 - val_accuracy: 0.7624
Epoch 3/5
552/552 [==============================] - 76s 137ms/step - loss: 0.4457 - accuracy: 0.8351 - val_loss: 0.4966 - val_accuracy: 0.7889
Epoch 4/5
552/552 [==============================] - 73s 133ms/step - loss: 0.3191 - accuracy: 0.8749 - val_loss: 0.7005 - val_accuracy: 0.7447
Epoch 5/5
552/552 [==============================] - 74s 135ms/step - loss: 0.2549 - accuracy: 0.9081 - val_loss: 0.5247 - val_accuracy: 0.7939


2024-02-04 21:17:13.208442: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 270901248 exceeds 10% of free system memory.


173/173 [==============================] - 3s 17ms/step
Accuracy of the SVM model (kernel: linear): 79.9709724238026
Accuracy of the SVM model (kernel: poly): 79.68069666182875
Accuracy of the SVM model (kernel: rbf): 79.95283018867924
